In [ ]:
import time

from coffea import hist
from coffea.analysis_objects import JaggedCandidateArray
import coffea.processor as processor
from functools import partial

from awkward import JaggedArray
import numpy as np


In [ ]:
# Look at ProcessorABC to see the expected methods and what they are supposed to do
class TTGammaProcessor(processor.ProcessorABC):
    def __init__(self, runNum = -1, eventNum = -1):
        dataset_axis = hist.Cat("dataset", "Primary dataset")
        mass_axis = hist.Bin("mass", r"$m_{\mu\mu}$ [GeV]", 600, 0., 600)
        pt_axis = hist.Bin("pt", r"$p_{T,\gamma}$ [GeV]", 300, 0., 300)
        mult_axis = hist.Bin("N", r"Multiplicity", 10, 0, 10)

        self._accumulator = processor.dict_accumulator({
            'pt_gamma': hist.Hist("Counts", dataset_axis, pt_axis),
            'pt_jet_pho': hist.Hist("Counts", dataset_axis, pt_axis),
            'pt_jet_pre': hist.Hist("Counts", dataset_axis, pt_axis),
            'jet_mult_pho': hist.Hist("Counts", dataset_axis, mult_axis),
            'jet_mult_pre': hist.Hist("Counts", dataset_axis, mult_axis),
            'M3': hist.Hist("Counts", dataset_axis, mass_axis),

            'mu_cutflow': processor.defaultdict_accumulator(partial(processor.defaultdict_accumulator, int)),
            'ele_cutflow': processor.defaultdict_accumulator(partial(processor.defaultdict_accumulator, int)),
            'eventList' : processor.defaultdict_accumulator(partial(processor.column_accumulator, np.zeros(shape=(0,)))),
            'runList' : processor.defaultdict_accumulator(partial(processor.column_accumulator, np.zeros(shape=(0,)))),
        })

        self.eventNum = eventNum
        self.runNum = runNum
        
    @property
    def accumulator(self):
        return self._accumulator

    def process(self, df):
        output = self.accumulator.identity()

        dataset = df['dataset']

        if '2016' in dataset:
            year=2016
            muonTrigger = df['HLT_IsoMu24'] | df['HLT_IsoTkMu24']
            eleTrigger = df['HLT_Ele27_WPTight_Gsf']
            photonBitMapName = 'Photon_cutBased'
        elif '2017' in dataset:
            year=2017
            muonTrigger = df['HLT_IsoMu27']
            eleTrigger = df['HLT_Ele32_WPTight_Gsf_L1DoubleEG'] | df['HLT_Ele32_WPTight_Gsf']
            photonBitMapName = 'Photon_cutBasedBitmap'
        elif '2018' in dataset:
            year=2018
            muonTrigger = df['HLT_IsoMu24']
            eleTrigger = df['HLT_Ele32_WPTight_Gsf']
            photonBitMapName = 'Photon_cutBasedBitmap'

        
        
        filters = (df['Flag_goodVertices'] &
                   df['Flag_globalSuperTightHalo2016Filter'] &
                   df['Flag_HBHENoiseFilter'] &
                   df['Flag_HBHENoiseIsoFilter'] &
                   df['Flag_EcalDeadCellTriggerPrimitiveFilter'] &
                   df['Flag_BadPFMuonFilter'] 
                  )
        if year > 2016:
            filters = (filters & 
                       df['Flag_ecalBadCalibFilterV2']
                      )
        
        
        muons = JaggedCandidateArray.candidatesfromcounts(
            df['nMuon'],
            pt=df['Muon_pt'],
            eta=df['Muon_eta'],
            phi=df['Muon_phi'],
            mass=df['Muon_mass'],
            charge=df['Muon_charge'],
            relIso=df['Muon_pfRelIso04_all'],
            tightId=df['Muon_tightId'],
            isPFcand=df['Muon_isPFcand'],
            isTracker=df['Muon_isTracker'],
            isGlobal=df['Muon_isGlobal'],           
            )
        
        electrons = JaggedCandidateArray.candidatesfromcounts(
            df['nElectron'],
            pt=df['Electron_pt'],
            eta=df['Electron_eta'],
            phi=df['Electron_phi'],
            mass=df['Electron_mass'],
            charge=df['Electron_charge'],
            cutBased=df['Electron_cutBased'],
            d0=df['Electron_dxy'],
            dz=df['Electron_dz'],
        )

        jets = JaggedCandidateArray.candidatesfromcounts(
            df['nJet'],
            pt=df['Jet_pt'],
            eta=df['Jet_eta'],
            phi=df['Jet_phi'],
            mass=df['Jet_mass'],
            jetId=df['Jet_jetId'],
            btag=df['Jet_btagDeepB'],
            hadFlav=df['Jet_hadronFlavour'],
            genIdx=df['Jet_genJetIdx'],
        )

        photons = JaggedCandidateArray.candidatesfromcounts(
            df['nPhoton'],
            pt=df['Photon_pt'],
            eta=df['Photon_eta'],
            phi=df['Photon_phi'],
            mass=df['Photon_mass'],
            isEE=df['Photon_isScEtaEE'],
            isEB=df['Photon_isScEtaEB'],
            photonId=df[photonBitMapName],
            passEleVeto=df['Photon_electronVeto'],
            pixelSeed=df['Photon_pixelSeed'],
        )
            

        muonSelectTight = ((muons.pt>30) & 
                           (abs(muons.eta)<2.4) & 
                           (muons.tightId) & 
                           (muons.relIso < 0.15)
                          )
        
        muonSelectLoose = ((muons.pt>15) & 
                           (abs(muons.eta)<2.4) & 
                           ((muons.isPFcand) & (muons.isTracker | muons.isGlobal)) & 
                           (muons.relIso < 0.25) &
                           np.invert(muonSelectTight)
                          )

        eleEtaGap = (abs(electrons.eta) < 1.4442) | (abs(electrons.eta) > 1.566)
        elePassD0 = ((abs(electrons.eta) < 1.479) & (abs(electrons.d0) < 0.05) |
                     (abs(electrons.eta) > 1.479)  & (abs(electrons.d0) < 0.1)
                    )
        elePassDZ = ((abs(electrons.eta) < 1.479) & (abs(electrons.dz) < 0.1) |
                     (abs(electrons.eta) > 1.479)  & (abs(electrons.dz) < 0.2)
                    )

        
        
        electronSelectTight = ((electrons.pt>35) & 
                               (abs(electrons.eta)<2.1) & 
#                                eleEtaGap &      
                               (electrons.cutBased>=4) &
                               elePassD0 & 
                               elePassDZ
                              )

        electronSelectLoose = ((electrons.pt>15) & 
                               (abs(electrons.eta)<2.4) & 
#                                eleEtaGap &      
                               (electrons.cutBased>=1) &
                               elePassD0 & 
                               elePassDZ & 
                               np.invert(electronSelectTight)
                              )
        
        tightMuon = muons[muonSelectTight]
        looseMuon = muons[muonSelectLoose]
        
        tightElectron = electrons[electronSelectTight]
        looseElectron = electrons[electronSelectLoose]

        

                
        oneMuon = (tightMuon.counts == 1)
        muVeto = (tightMuon.counts == 0)
        oneEle = (tightElectron.counts == 1)
        eleVeto = (tightElectron.counts == 0)
        looseMuonSel = (looseMuon.counts == 0)
        looseElectronSel = (looseElectron.counts == 0)

        
        #### Calculate deltaR between photon and nearest muon
        ####### make combination pairs
        phoMu = photons['p4'].cross(tightMuon['p4'],nested=True)
        ####### check delta R of each combination, if min is >0.1 it is okay, or if there are no tight muons it passes
        dRphomu = (phoMu.i0.delta_r(phoMu.i1)>0.4).all() | (tightMuon.counts==0)
        phoEle = photons['p4'].cross(tightElectron['p4'],nested=True)
        dRphoele = ((phoEle.i0.delta_r(phoEle.i1)).min()>0.4) | (tightElectron.counts==0)
        
        photonSelect = ((photons.pt>20) & 
                        (abs(photons.eta) < 1.4442) &
                        (photons.isEE | photons.isEB) &
                        ((photons.photonId >> 1 & 1) == 1) & 
                        (photons.passEleVeto) & 
                        np.invert(photons.pixelSeed) & 
                        dRphomu & dRphoele
                       )
        tightPhotons = photons[photonSelect]
        
        jetIDbit = 1
        if year>2016: jetIDbit=2

        jetMu = jets['p4'].cross(tightMuon['p4'],nested=True)
        dRjetmu = ((jetMu.i0.delta_r(jetMu.i1)).min()>0.4) | (tightMuon.counts==0)

        jetEle = jets['p4'].cross(tightElectron['p4'],nested=True)
        dRjetele = ((jetEle.i0.delta_r(jetEle.i1)).min()>0.4) | (tightElectron.counts==0)

        jetPho = jets['p4'].cross(tightPhotons['p4'],nested=True)
        dRjetpho = ((jetPho.i0.delta_r(jetPho.i1)).min()>0.1) | (tightPhotons.counts==0)
        
        jetSelect = ((jets.pt > 30) &
                     (abs(jets.eta) < 2.4) &
                     ((jets.jetId >> jetIDbit & 1)==1) &
                     dRjetmu & dRjetele & dRjetpho                    
                    )

        tightJets = jets[jetSelect]
        
        bTagWP = 0.6321
        if year == 2017:
            bTagWP = 0.4941
        if year == 2018:
            bTagWP = 0.4184
        bJets = jets[jetSelect & (jets.btag>bTagWP)]

        if self.eventNum >= 0 and self.runNum >= 0:
            singleEvent = (df['run']==self.runNum) & (df['event']==self.eventNum)
            muonTrigger = muonTrigger & singleEvent
            eleTrigger = eleTrigger & singleEvent
        
        mu_trigger = muonTrigger        
        mu_filter = mu_trigger# & filters
        mu_oneMu = oneMuon & mu_filter
        mu_noEle = eleVeto & mu_oneMu
        mu_noLoose = looseMuonSel & looseElectronSel & mu_noEle
        mu_twoJet = (tightJets.counts >= 2) & mu_noLoose
        mu_threeJet = (tightJets.counts >= 3) & mu_noLoose
        mu_jetSel = (tightJets.counts >= 4) & mu_noLoose
        mu_bjetSel = (bJets.counts >= 1) & mu_jetSel
        mu_presel = mu_bjetSel
        mu_phosel = (tightPhotons.counts >= 1) & mu_bjetSel

        ele_trigger = eleTrigger
        ele_filter = ele_trigger# & filters
        ele_oneEle = oneEle & ele_filter
        ele_noMu = muVeto & ele_oneEle
        ele_noLoose = looseMuonSel & looseElectronSel & ele_noMu
        ele_twoJet = (tightJets.counts >= 2) & ele_noLoose
        ele_threeJet = (tightJets.counts >= 3) & ele_noLoose
        ele_jetSel = (tightJets.counts >= 4) & ele_noLoose
        ele_bjetSel = (bJets.counts >= 1) & ele_jetSel
        ele_presel = ele_bjetSel
        ele_phosel = (tightPhotons.counts >= 1) & ele_bjetSel
        
        output['mu_cutflow'][dataset]['all events'] += mu_trigger.size
        output['mu_cutflow'][dataset]['trigger'] += mu_trigger.sum()
        output['mu_cutflow'][dataset]['filter'] += mu_filter.sum()
        output['mu_cutflow'][dataset]['lepSel'] += mu_oneMu.sum()
        output['mu_cutflow'][dataset]['lepVeto'] += mu_noEle.sum()
        output['mu_cutflow'][dataset]['noLooseLep'] += mu_noLoose.sum()
        output['mu_cutflow'][dataset]['twoJets'] += mu_twoJet.sum()
        output['mu_cutflow'][dataset]['threeJets'] += mu_threeJet.sum()
        output['mu_cutflow'][dataset]['fourJets'] += mu_jetSel.sum()
        output['mu_cutflow'][dataset]['bTag'] += mu_bjetSel.sum()
        output['mu_cutflow'][dataset]['phosel'] += mu_phosel.sum()

    
    
        output['ele_cutflow'][dataset]['all events'] += ele_trigger.size
        output['ele_cutflow'][dataset]['trigger'] += ele_trigger.sum()
        output['ele_cutflow'][dataset]['filter'] += ele_filter.sum()
        output['ele_cutflow'][dataset]['lepSel'] += ele_oneEle.sum()
        output['ele_cutflow'][dataset]['lepVeto'] += ele_noMu.sum()
        output['ele_cutflow'][dataset]['noLooseLep'] += ele_noLoose.sum()
        output['ele_cutflow'][dataset]['twoJets'] += ele_twoJet.sum()
        output['ele_cutflow'][dataset]['threeJets'] += ele_threeJet.sum()
        output['ele_cutflow'][dataset]['fourJets'] += ele_jetSel.sum()
        output['ele_cutflow'][dataset]['bTag'] += ele_bjetSel.sum()
        output['ele_cutflow'][dataset]['phosel'] += ele_phosel.sum()
    
                
        output['pt_gamma'].fill(dataset=dataset,
                            pt=tightPhotons.p4.pt[:,:1][mu_phosel].flatten())
        output['pt_jet_pho'].fill(dataset=dataset,
                            pt=tightJets.p4.pt[:,:1][mu_phosel].flatten())
        output['pt_jet_pre'].fill(dataset=dataset,
                            pt=tightJets.p4.pt[:,:1][mu_presel].flatten())
        output['jet_mult_pho'].fill(dataset=dataset,
                                N=tightJets[mu_phosel].counts.flatten())
        output['jet_mult_pre'].fill(dataset=dataset,
                                N=tightJets[mu_presel].counts.flatten())
        
        if self.eventNum >= 0 and singleEvent.sum()>0 :
            print ("Muons")
            print ("  pt    ",muons[singleEvent].pt)
            print ("  eta   ",muons[singleEvent].eta)
            print ("  id    ",muons[singleEvent].tightId)
            print ("  iso   ",muons[singleEvent].relIso)
            print ("  sel   ",muonSelectTight[singleEvent])
            print ("  loose ",muonSelectLoose[singleEvent])

            print ('-'*20)
            print ("Electrons")
            print ("  pt    ",electrons[singleEvent].pt)
            print ("  eta   ",electrons[singleEvent].eta)
            print ("  id    ",electrons[singleEvent].cutBased)
            print ("  gap   ",eleEtaGap[singleEvent])
            print ("  d0    ",electrons[singleEvent].d0)
            print ("  passd0",elePassD0[singleEvent])
            print ("  dz    ",electrons[singleEvent].dz)
            print ("  passdz",elePassDZ[singleEvent])
            print ("  sel   ",electronSelectTight[singleEvent])
            print ("  loose ",electronSelectLoose[singleEvent])

            print ('-'*20)
            print ("Photons")
            print ("  pt     ",photons[singleEvent].pt)
            print ("  eta    ",photons[singleEvent].eta)
            print ("  id     ",photons[singleEvent].photonId)
            print ("  eVeto  ",photons[singleEvent].passEleVeto)
            print ("  pSeed  ",np.invert(photons[singleEvent].pixelSeed))
            print ("  dRmu   ",dRphomu[singleEvent])
            print ("  dRmu   ",(phoMu.i0.delta_r(phoMu.i1))[singleEvent])
            print ("  dRe    ",dRphoele[singleEvent])
            print ("  dRe    ",(phoEle.i0.delta_r(phoEle.i1))[singleEvent])
            print ("  select ",photonSelect[singleEvent])


        output['runList'][dataset+'_mu_phosel'] += processor.column_accumulator(df['run'][mu_phosel].flatten())    
        output['eventList'][dataset+'_mu_phosel'] += processor.column_accumulator(df['event'][mu_phosel].flatten())    
        output['runList'][dataset+'_mu_presel'] += processor.column_accumulator(df['run'][mu_presel].flatten())    
        output['eventList'][dataset+'_mu_presel'] += processor.column_accumulator(df['event'][mu_presel].flatten())    

        output['runList'][dataset+'_ele_phosel'] += processor.column_accumulator(df['run'][ele_phosel].flatten())    
        output['eventList'][dataset+'_ele_phosel'] += processor.column_accumulator(df['event'][ele_phosel].flatten())    
        output['runList'][dataset+'_ele_presel'] += processor.column_accumulator(df['run'][ele_presel].flatten())    
        output['eventList'][dataset+'_ele_presel'] += processor.column_accumulator(df['event'][ele_presel].flatten())    
        
        return output

    def postprocess(self, accumulator):
        return accumulator


In [ ]:
tstart = time.time()

fileset = {
    'TTGamma2016_NoFullyHad': [
        'Files/myNanoProdMc2016_job0.root',
        'Files/myNanoProdMc2016_job1.root',
        'Files/myNanoProdMc2016_job2.root',
        'Files/myNanoProdMc2016_job3.root',
        'Files/myNanoProdMc2016_job4.root',
        'Files/myNanoProdMc2016_job5.root',
        'Files/myNanoProdMc2016_job6.root',
        'Files/myNanoProdMc2016_job7.root',
        'Files/myNanoProdMc2016_job8.root',
        'Files/myNanoProdMc2016_job9.root',
    ],
    'TTGamma2017_NoFullyHad': [
        'Files/myNanoProdMc2017_job0.root',
        'Files/myNanoProdMc2017_job1.root',
        'Files/myNanoProdMc2017_job2.root',
        'Files/myNanoProdMc2017_job3.root',
        'Files/myNanoProdMc2017_job4.root',
        'Files/myNanoProdMc2017_job5.root',
        'Files/myNanoProdMc2017_job6.root',
        'Files/myNanoProdMc2017_job7.root',
        'Files/myNanoProdMc2017_job8.root',
        'Files/myNanoProdMc2017_job9.root',
    ],
    'TTGamma2018_NoFullyHad': [
        'Files/myNanoProdMc2018_job0.root',
        'Files/myNanoProdMc2018_job1.root',
        'Files/myNanoProdMc2018_job2.root',
        'Files/myNanoProdMc2018_job3.root',
        'Files/myNanoProdMc2018_job4.root',
        'Files/myNanoProdMc2018_job5.root',
        'Files/myNanoProdMc2018_job6.root',
        'Files/myNanoProdMc2018_job7.root',
        'Files/myNanoProdMc2018_job8.root',
        'Files/myNanoProdMc2018_job9.root',
    ],
}

output = processor.run_uproot_job(fileset,
                                  treename='Events',
                                  processor_instance=TTGammaProcessor(),
                                  executor=processor.futures_executor,
                                  executor_args={'workers': 6, 'flatten': True},
                                  chunksize=100000,
                                 )

elapsed = time.time() - tstart
for k in fileset.keys():
    print(k,output['mu_cutflow'][k])
    print(k,output['ele_cutflow'][k])
print(elapsed)

In [ ]:
import uproot

_tree2016 = uproot.open('/uscms/home/dnoonan/work/13TeV_TTGamma/NanoAODTest/SynchWithNabin/TTGammaSemiLep_13TeV/TTGamma_noFullyHad_2016_AnalysisNtuple.root')['AnalysisTree']
_tree2017 = uproot.open('/uscms/home/dnoonan/work/13TeV_TTGamma/NanoAODTest/SynchWithNabin/TTGammaSemiLep_13TeV/TTGamma_noFullyHad_2017_AnalysisNtuple.root')['AnalysisTree']
_tree2018 = uproot.open('/uscms/home/dnoonan/work/13TeV_TTGamma/NanoAODTest/SynchWithNabin/TTGammaSemiLep_13TeV/TTGamma_noFullyHad_2018_AnalysisNtuple.root')['AnalysisTree']


_tree = _tree2016
runs = _tree.array('run')
event = _tree.array('event')
lepSel = (_tree.array('nMu')==1)
jetSel = (_tree.array('nJet')>=4) & lepSel
bjetSel = (_tree.array('nBJet')>=1) & jetSel
phoSel = (_tree.array('nPhoBarrel')>=1) & bjetSel
eventList = np.array([runs[phoSel],event[phoSel]]).transpose()
mu_eventList2016= [list(x) for x in eventList]

_tree = _tree2017
runs = _tree.array('run')
event = _tree.array('event')
lepSel = (_tree.array('nMu')==1)
jetSel = (_tree.array('nJet')>=4) & lepSel
bjetSel = (_tree.array('nBJet')>=1) & jetSel
phoSel = (_tree.array('nPhoBarrel')>=1) & bjetSel
eventList = np.array([runs[phoSel],event[phoSel]]).transpose()
mu_eventList2017= [list(x) for x in eventList]

_tree = _tree2018
runs = _tree.array('run')
event = _tree.array('event')
lepSel = (_tree.array('nMu')==1)
jetSel = (_tree.array('nJet')>=4) & lepSel
bjetSel = (_tree.array('nBJet')>=1) & jetSel
phoSel = (_tree.array('nPhoBarrel')>=1) & bjetSel
eventList = np.array([runs[phoSel],event[phoSel]]).transpose()
mu_eventList2018= [list(x) for x in eventList]

_tree = _tree2016
runs = _tree.array('run')
event = _tree.array('event')
lepSel = (_tree.array('nEle')==1)
jetSel = (_tree.array('nJet')>=4) & lepSel
bjetSel = (_tree.array('nBJet')>=1) & jetSel
phoSel = (_tree.array('nPhoBarrel')>=1) & bjetSel
eventList = np.array([runs[phoSel],event[phoSel]]).transpose()
ele_eventList2016= [list(x) for x in eventList]

_tree = _tree2017
runs = _tree.array('run')
event = _tree.array('event')
lepSel = (_tree.array('nEle')==1)
jetSel = (_tree.array('nJet')>=4) & lepSel
bjetSel = (_tree.array('nBJet')>=1) & jetSel
phoSel = (_tree.array('nPhoBarrel')>=1) & bjetSel
eventList = np.array([runs[phoSel],event[phoSel]]).transpose()
ele_eventList2017= [list(x) for x in eventList]

_tree = _tree2018
runs = _tree.array('run')
event = _tree.array('event')
lepSel = (_tree.array('nEle')==1)
jetSel = (_tree.array('nJet')>=4) & lepSel
bjetSel = (_tree.array('nBJet')>=1) & jetSel
phoSel = (_tree.array('nPhoBarrel')>=1) & bjetSel
eventList = np.array([runs[phoSel],event[phoSel]]).transpose()
ele_eventList2018= [list(x) for x in eventList]

In [ ]:
b = np.array([output['runList']['TTGamma2016_NoFullyHad_ele_phosel'].value,output['eventList']['TTGamma2016_NoFullyHad_ele_phosel'].value]).transpose()
b = [list(x) for x in b]

print(len(b))
print(len(ele_eventList2016))
print("Me and not you")
print([x for x in b if not x in ele_eventList2016])
print("You and not me")
print([x for x in ele_eventList2016 if not x in b])


In [ ]:
b = np.array([output['runList']['TTGamma2017_NoFullyHad_ele_phosel'].value,output['eventList']['TTGamma2017_NoFullyHad_ele_phosel'].value]).transpose()
b = [list(x) for x in b]

print(len(b))
print(len(ele_eventList2017))
print("Me and not you")
print([x for x in b if not x in ele_eventList2017])
print("You and not me")
print([x for x in ele_eventList2017 if not x in b])


In [ ]:
b = np.array([output['runList']['TTGamma2018_NoFullyHad_ele_phosel'].value,output['eventList']['TTGamma2018_NoFullyHad_ele_phosel'].value]).transpose()
b = [list(x) for x in b]

print(len(b))
print(len(ele_eventList2018))
print("Me and not you")
print([x for x in b if not x in ele_eventList2018])
print("You and not me")
print([x for x in ele_eventList2018 if not x in b])


In [ ]:
b = np.array([output['runList']['TTGamma2016_NoFullyHad_mu_phosel'].value,output['eventList']['TTGamma2016_NoFullyHad_mu_phosel'].value]).transpose()
b = [list(x) for x in b]

print(len(b))
print(len(mu_eventList2016))
print("Me and not you")
print([x for x in b if not x in mu_eventList2016])
print("You and not me")
print([x for x in mu_eventList2016 if not x in b])


In [ ]:
b = np.array([output['runList']['TTGamma2017_NoFullyHad_mu_phosel'].value,output['eventList']['TTGamma2017_NoFullyHad_mu_phosel'].value]).transpose()
b = [list(x) for x in b]

print(len(b))
print(len(mu_eventList2017))
print("Me and not you")
print([x for x in b if not x in mu_eventList2017])
print("You and not me")
print([x for x in mu_eventList2017 if not x in b])


In [ ]:
b = np.array([output['runList']['TTGamma2018_NoFullyHad_mu_phosel'].value,output['eventList']['TTGamma2018_NoFullyHad_mu_phosel'].value]).transpose()
b = [list(x) for x in b]

print(len(b))
print(len(mu_eventList2018))
print("Me and not you")
print([x for x in b if not x in mu_eventList2018])
print("You and not me")
print([x for x in mu_eventList2018 if not x in b])


In [ ]:
# sel = ['all events', 'trigger', 'filter', 'lepSel', 'lepVeto', 'noLooseLep', 'fourJets', 'bTag','phosel']
sel = {'all events': "Entries", 
       'trigger': "Trigger Cut", 
       'filter': "Filter", 
       'lepSel': "One muon", 
       'lepVeto': "Lep Veto", 
       'noLooseLep': "Loose Veto", 
       'fourJets': "$\\geq 4$ jets", 
       'bTag': "$\\geq 1$ b-jet",
       'phosel' : "$\\geq 1$ photon"
      }

print("\\begin{tabular}{l | c | c | c }")
print("\\hline")
print("Cut & 2016 & 2017 & 2018 \\\\")
print("\\hline")
for s,v in sel.items():
    print("%s & %i & %i & %i \\\\"%(v,output['mu_cutflow']['TTGamma2016_NoFullyHad'][s],output['mu_cutflow']['TTGamma2017_NoFullyHad'][s],output['mu_cutflow']['TTGamma2018_NoFullyHad'][s]))
print("\\hline")
print("\\end{tabular}")


In [ ]:
# sel = ['all events', 'trigger', 'filter', 'lepSel', 'lepVeto', 'noLooseLep', 'fourJets', 'bTag','phosel']
sel = {'all events': "Entries", 
       'trigger': "Trigger Cut", 
       'filter': "Filter", 
       'lepSel': "One electron", 
       'lepVeto': "Lep Veto", 
       'noLooseLep': "Loose Veto", 
       'fourJets': "$\\geq 4$ jets", 
       'bTag': "$\\geq 1$ b-jet",
       'phosel' : "$\\geq 1$ photon"
      }

print("\\begin{tabular}{l | c | c | c }")
print("\\hline")
print("Cut & 2016 & 2017 & 2018 \\\\")
print("\\hline")
for s,v in sel.items():
    print("%s & %i & %i & %i \\\\"%(v,output['ele_cutflow']['TTGamma2016_NoFullyHad'][s],output['ele_cutflow']['TTGamma2017_NoFullyHad'][s],output['ele_cutflow']['TTGamma2018_NoFullyHad'][s]))
print("\\hline")
print("\\end{tabular}")


In [ ]:
# fig, ax, _ = hist.plot1d(output['pt_jet_pho'], overlay='dataset')
# ax.set_xlim(0,250)
